# Build a Kriging surrogate on some dummy prop data

In [273]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import dummy_prop_example
from smt.sampling_methods import FullFactorial, LHS
from smt.surrogate_models import KRG, KPLS, KPLSK
from smt.applications import MFK
from sklearn.model_selection import train_test_split 
import sys
from pathlib import Path
sys.path[0] = str(Path(sys.path[0]).parent)
# from unipy import surrogate_model

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**First let us consider a 'low fidelity' dataset. We could build a full factorial of data if this is cheap, so lets pretend to do that.**

We also may choose to remove some cases, since we can't always successfully manage to build a full factorial set of structured data. 

Let us set up the sampling limits:

*Note:* that we use the kwarg `clip=True` in order for the number of points to give a full grid.

In [108]:
# setup each limit
discangle_limits = [-90.0, 90.0]
propspeed_limits = [400.0, 1200.0]
airspeed_limits = [0.0, 60.0]
# and we group them into a list
xlimits = np.array([discangle_limits, airspeed_limits, propspeed_limits])
# and make the full factorial sampling
lf_sampling = FullFactorial(xlimits=xlimits, clip=True)
number_of_lf_samples = 200
lf_independent = lf_sampling(number_of_lf_samples)
print(f"Actual number of samples generated is: {lf_independent[:, 0].shape[0]}")

Actual number of samples generated is: 216


**now we can plot the inputs, just to see what we have achieved**

In [109]:
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=lf_independent[:, 0],
            y=lf_independent[:, 1],
            z=lf_independent[:, 2],
            mode="markers",
            marker=dict(color="black", size=4),
        )
    ]
)

fig.update_layout(
    scene=dict(
        xaxis_title="disc angle [deg]",
        yaxis_title="airspeed [m/s]",
        zaxis_title="prop speed [RPM]",
    ),
)

fig.show()


**It is now time to build a Pandas DataFrame, containg all of this data**

We don't really need to do this to build a Kirging surrogate, however we are doing this as it is common that the data won't just come out as a tidy NumPy array, and instead will be loads from some .csv file or similar.

*Note:* that the arguments after the sampling points are used to caluclate the dependent variable and are unique to the propeller example we are running.

In [110]:
lf_data_df: pd.DataFrame = dummy_prop_example.lf_data(
    lf_independent[:, 0],
    lf_independent[:, 1],
    lf_independent[:, 2],
    1000.0,
    4000.0,
)

display(lf_data_df)

,airspeed,discangle,propspeed,load
0,0.0,-90.0,400.0,640.0
1,0.0,-90.0,560.0,1254.4
2,0.0,-90.0,720.0,2073.6
3,0.0,-90.0,880.0,3097.6
4,0.0,-90.0,1040.0,4326.4
...,...,...,...,...
211,60.0,90.0,560.0,12774.4
212,60.0,90.0,720.0,13593.6
213,60.0,90.0,880.0,14617.6
214,60.0,90.0,1040.0,15846.4


**At this stage, it is probably worthwhile to see what the data looks like**

*Note:* that this data and the trends are made up, the trends are also made up

In [111]:
rpms = np.unique(lf_data_df.propspeed.to_numpy())
rpm = rpms[0]
plot_trend_df = lf_data_df[lf_data_df.propspeed == rpm]

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=plot_trend_df.discangle,
            y=plot_trend_df.airspeed,
            z=plot_trend_df.load,
            mode="markers",
            marker=dict(
                color="black", size=4),
        )
    ]
)

fig.update_layout(
    scene=dict(
        xaxis_title="disc angle [deg]",
        yaxis_title="airspeed [m/s]",
        zaxis_title="thrust [N]",
    ),
    title=f"Prop speed is {rpm} RPM",
)


**Now we can build the Kirging surrogate model, from the DataFrame**

In [112]:
# we have yet to explore theta0, using default from docs
krg_sm = KRG(theta0=[1e-2])

**Lets now write a function which can split the data using train-test split from sklearn**

We probably wouldn't use this for the low-fidelity data, but it might come in useful later anyway. 

*Note:* that the purpose here is to give some understanding before the *proper* code is written. This is just a first looksee at what we could do.

In [113]:
def prep_data(
    df: pd.DataFrame, headers: list[str]
) -> np.ndarray:
    """ Prepare the data for surrogate model
    Prepares an np.ndarray[nt, nx] for the Kirger, where nx is in
    the order of the specified headers
    """
    return np.asarray([df[h].to_numpy() for h in headers]).T


**Now let us make the Kirging, without any thought to validation**

In [114]:
lf_x_data = prep_data(lf_data_df, ["discangle", "airspeed", "propspeed"])
lf_y_data = prep_data(lf_data_df, ["load"])
krg_sm.set_training_values(lf_x_data, lf_y_data)

**Now we train the model**

In [115]:
krg_sm.train()

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 216
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  1.5425088


**Now let us make some points for interpolation**

In [253]:
# I like to use non-equal dimensions, it helps to make sure plotting is correct!
# n_propspeed = 20
n_discangle_lin = 22
n_airspeed_lin = 21
single_rpm = lf_data_df[lf_data_df.propspeed == propspeed_limits[0]]
# we use `*_vec_i` to hint that it's a 1D vector or interp points
discangle_vec_i = np.sort(np.concatenate(
    [np.linspace(-90, 90, n_discangle_lin), single_rpm.discangle.to_numpy()]
))
airspeed_vec_i = np.sort(np.concatenate(
    [np.linspace(0, 60, n_airspeed_lin), single_rpm.airspeed.to_numpy()]
))

# now the actual number of points
n_discangle = len(discangle_vec_i)
n_airspeed = len(airspeed_vec_i)

plot_rpm = rpms[0]
propspeed_vec_i = np.asarray(plot_rpm)
# and now we mesh grid, so that we can make a surface
# note that we use `*_mat_i` to show that it's not longer a vector
[discangle_mat_i, airspeed_mat_i, propspeed_mat_i] = np.meshgrid(
    discangle_vec_i, airspeed_vec_i, propspeed_vec_i
)

# data to interpolate
x_data_interp = np.asarray(
    [
        discangle_mat_i.flatten(),
        airspeed_mat_i.flatten(),
        propspeed_mat_i.flatten(),
    ]
).T
print(f"Shape of the data to interpolate is {x_data_interp.shape}")

y_data_interp = krg_sm.predict_values(x_data_interp)
y_data_s2 = krg_sm.predict_variances(x_data_interp)


Shape of the data to interpolate is (3306, 3)
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 3306
   
   Predicting ...
   Predicting - done. Time (sec):  0.2517090
   
   Prediction time/pt. (sec) :  0.0000761
   


**Now we can finally plot the data**

*Note:* that we first need to reshape the data

In [254]:
y_data_interp_plt = y_data_interp.reshape(n_airspeed, n_discangle)
y_data_s2_plt = y_data_s2.reshape(n_airspeed, n_discangle)

print(y_data_interp_plt.shape)
print(airspeed_mat_i[:, :, 0].shape)
print(discangle_mat_i[:, :, 0].shape)


(57, 58)
(57, 58)
(57, 58)


In [258]:
fig = go.Figure(
    data=[
        go.Surface(
            z=y_data_interp_plt, y=airspeed_mat_i[:, :, 0], x=discangle_mat_i[:, :, 0]
        ),
        # go.Surface(
        #     z=y_data_interp_plt + 300 * np.sqrt(y_data_s2_plt),
        #     y=airspeed_mat_i[:, :, 0],
        #     x=discangle_mat_i[:, :, 0],
        # ),
        go.Scatter3d(
            x=plot_trend_df.discangle,
            y=plot_trend_df.airspeed,
            z=plot_trend_df.load,
            mode="markers",
            marker=dict(color="black", size=4),
        ),
    ]
)

fig.update_layout(
    title=f"Interp at prop speed = {plot_rpm} RPM",
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=65, r=50, b=65, t=90),
)

fig.show()


**We can also look at the variance of the fit**

Although it should be noted that the variance is likely down to the sampling used, not just the confidence the Kirging model has. Let's make a plot anyway, and see what values fall within two standard deviations $(2\sigma)$ 

**Now let us add some noise in**

We will do there under cases where the axial advance ratio is high

In [249]:
lf_noise_krg_sm = KRG(theta0=[1e-2], eval_noise=True)
lf_noise_df = dummy_prop_example.add_noise_adv_rat(lf_data_df, 0.11)
lf_noise_x_data = prep_data(lf_noise_df, ["discangle", "airspeed", "propspeed"])
lf_noise_y_data = prep_data(lf_noise_df, ["load_noise"])
lf_noise_krg_sm.set_training_values(lf_noise_x_data, lf_noise_y_data)
lf_noise_krg_sm.train()


___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 216
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  9.2023153


In [259]:
noise_y_data_interp = lf_noise_krg_sm.predict_values(x_data_interp)
noise_variance = lf_noise_krg_sm.predict_variances(x_data_interp)

noise_y_data_interp_plt = noise_y_data_interp.reshape(n_airspeed, n_discangle)
noise_variance_plt = noise_variance.reshape(n_airspeed, n_discangle)

___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 3306
   
   Predicting ...
   Predicting - done. Time (sec):  0.1255491
   
   Prediction time/pt. (sec) :  0.0000380
   


In [261]:
rpm = 400
noise_plot_trend_df = lf_noise_df[lf_noise_df.propspeed == rpm]
fig = go.Figure(
    data=[
        go.Surface(
            z=noise_y_data_interp_plt,
            y=airspeed_mat_i[:, :, 0],
            x=discangle_mat_i[:, :, 0],
        ),
        # go.Surface(
        #     z=noise_y_data_interp_plt + 3 * np.sqrt(noise_variance_plt),
        #     y=airspeed_mat_i[:, :, 0],
        #     x=discangle_mat_i[:, :, 0],
        # ),
        go.Scatter3d(
            x=noise_plot_trend_df.discangle,
            y=noise_plot_trend_df.airspeed,
            z=noise_plot_trend_df.load_noise,
            mode="markers",
            marker=dict(color="black", size=4),
        ),
        go.Scatter3d(
            x=noise_plot_trend_df.discangle,
            y=noise_plot_trend_df.airspeed,
            z=noise_plot_trend_df.load,
            mode="markers",
            marker=dict(color="red", size=4),
        ),
    ]
)

fig.update_layout(
    title=f"Interp at prop speed = {plot_rpm} RPM",
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=65, r=50, b=65, t=90),
)

fig.show()


**From the point of view of fitting, this is a very simple example.**

We have a wonderfully smooth funciton, gridded data (we could even find good resukts with 'ndinterp' or similar), and there is zero noise. In reality we will likely seem some zones which require some smoothing, either due to noise in measurements, or solvers struggling with challenging cases. We will not produce a 'High fidelity data set', which again will be manafactured, however we will put in some noise and some mild discontinuities in the function, perhaps something that will represent stall.

**Let's generate the 'high fidelity' data**

In [267]:
ff_hf_data_df: pd.DataFrame = dummy_prop_example.hf_data(
    lf_independent[:, 0],
    lf_independent[:, 1],
    lf_independent[:, 2],
    1000.0,
    3700.0,
)

display(ff_hf_data_df)

,airspeed,discangle,propspeed,load
0,0.0,-90.0,400.0,562.400000
1,0.0,-90.0,560.0,1102.304000
2,0.0,-90.0,720.0,1822.176000
3,0.0,-90.0,880.0,2722.016000
4,0.0,-90.0,1040.0,3801.824000
...,...,...,...,...
211,60.0,90.0,560.0,20674.548898
212,60.0,90.0,720.0,17045.033143
213,60.0,90.0,880.0,15177.080935
214,60.0,90.0,1040.0,14340.725099


**Now lets compare the two datasets, at a common, fixed, propeller speed**

In [283]:
rpms = np.unique(lf_data_df.propspeed.to_numpy())
rpm = rpms[0]
lf_plot_trend_df = lf_data_df[lf_data_df.propspeed == rpm]
hf_plot_trend_df = ff_hf_data_df[ff_hf_data_df.propspeed == rpm]

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=noise_plot_trend_df.discangle,
            y=noise_plot_trend_df.airspeed,
            z=noise_plot_trend_df.load_noise,
            mode="markers",
            marker=dict(color="black", size=4),
            name="low fidelity",
        ),
        go.Scatter3d(
            x=hf_plot_trend_df.discangle,
            y=hf_plot_trend_df.airspeed,
            z=hf_plot_trend_df.load,
            mode="markers",
            marker=dict(color="blue", size=4),
            name="high fidelity",
        ),
    ]
)

fig.update_layout(
    scene=dict(
        xaxis_title="disc angle [deg]",
        yaxis_title="airspeed [m/s]",
        zaxis_title="thrust [N]",
    ),
    title=f"Prop speed is {rpm} RPM",
)


**Now let's generate fewer high-fidelity points**

This time we will use a Latin Hypercube sample

In [308]:
hf_lhs = LHS(xlimits=xlimits)
number_of_lf_samples = 40
lhs_hf_independent = hf_lhs(number_of_lf_samples)

Now let us calculate the data at each LHS point, note that we won't add noise to the HF data, we assume this data is of higher quality. We can add noise if we like though, it will still be handled.

In [309]:
lhs_hf_data_df: pd.DataFrame = dummy_prop_example.hf_data(
    lhs_hf_independent[:, 0],
    lhs_hf_independent[:, 1],
    lhs_hf_independent[:, 2],
    1000.0,
    3700.0,
)


**We can now begin to cosider co-Kriging, where we can use the high fidelity data to improve the low-fidelity model**

Let's sort the training data (again in the case of the noisy lf data)

In [310]:
lf_noise_x_data = prep_data(lf_noise_df, ["discangle", "airspeed", "propspeed"])
lf_noise_y_data = prep_data(lf_noise_df, ["load_noise"])

hf_x_data = prep_data(lhs_hf_data_df, ["discangle", "airspeed", "propspeed"])
hf_y_data = prep_data(lhs_hf_data_df, ["load"])

Now we can make the MFK model

In [311]:
sm = MFK(theta0=[1e-2], eval_noise=True)
# low-fidelity dataset names being integers from 0 to level-1
sm.set_training_values(lf_noise_x_data, lf_noise_y_data, name=0)
# high-fidelity dataset without name
sm.set_training_values(hf_x_data, hf_y_data)
# train the model
sm.train()

___________________________________________________________________________
   
                                    MFK
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 40
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  2.7353654


Now we predict the gird, as before

In [312]:
mfk_vec_interp = sm.predict_values(x_data_interp)
mfk_mat_plt = mfk_vec_interp.reshape(n_airspeed, n_discangle)

___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 3306
   
   Predicting ...
   Predicting - done. Time (sec):  0.0895402
   
   Prediction time/pt. (sec) :  0.0000271
   


Now we can plot

In [313]:
noise_plot_trend_df = lf_noise_df[lf_noise_df.propspeed == rpm]
fig = go.Figure(
    data=[
        go.Surface(
            z=mfk_mat_plt,
            y=airspeed_mat_i[:, :, 0],
            x=discangle_mat_i[:, :, 0],
        ),
        go.Scatter3d(
            x=noise_plot_trend_df.discangle,
            y=noise_plot_trend_df.airspeed,
            z=noise_plot_trend_df.load_noise,
            mode="markers",
            marker=dict(color="black", size=4),
        ),
        go.Scatter3d(
            x=noise_plot_trend_df.discangle,
            y=noise_plot_trend_df.airspeed,
            z=noise_plot_trend_df.load,
            mode="markers",
            marker=dict(color="red", size=4),
        ),
        go.Scatter3d(
            x=hf_plot_trend_df.discangle,
            y=hf_plot_trend_df.airspeed,
            z=hf_plot_trend_df.load,
            mode="markers",
            marker=dict(color="blue", size=4),
            name="high fidelity",
        ),
    ]
)

fig.update_layout(
    title=f"Interp at prop speed = {plot_rpm} RPM",
    autosize=False,
    width=500,
    height=500,
    margin=dict(l=65, r=50, b=65, t=90),
)

fig.show()
